In [1]:
import os
from glob import glob
import onnx

import onnxruntime
from tqdm.notebook import tqdm

In [2]:
[0, 1] + [2**i for i in range(2, 9)]

[0, 1, 4, 8, 16, 32, 64, 128, 256]

In [3]:
import cv2
import os
import torch
import json
import numpy as np
from tqdm.notebook import tqdm

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from mmcv.transforms import Compose
from mmdet.utils import get_test_pipeline_cfg

def read_json(json_path):
    with open(json_path) as f:
        data = json.load(f)
    return data

def read_txt(txt_path):
    with open(txt_path) as f:
        data = f.readlines()
    data = [x.strip() for x in data]
    return data

def preprocess(test_pipeline, image):
    if isinstance(image, np.ndarray):
        # Calling this method across libraries will result
        # in module unregistered error if not prefixed with mmdet.
        test_pipeline[0].type = 'mmdet.LoadImageFromNDArray'
    test_pipeline = Compose(test_pipeline)
    return test_pipeline(dict(img=image))

class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, annotations_json_path, transform=None):
        self.transform = transform
        self.images_dir = images_dir
        self.annotations_json = read_json(annotations_json_path)


    def __len__(self):
        return len(self.annotations_json['images'])

    def __getitem__(self, idx):
        image_dict = self.annotations_json['images'][idx]
        image_path = os.path.join(self.images_dir, image_dict['file_name'])
        image_id = image_dict['id']

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            transformed_images = self.transform(image)
        else:
            transformed_images = image

        return image_id, image_path, transformed_images


# calibrationDataloader = DataLoader(calibrationDataset, batch_size=32, shuffle=True)
import torch
from mmdet.apis import DetInferencer

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize([640, 640]),  # Resize
])

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CONFIG_PATH = '/teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco.py'
WEIGHTS_PATH = '/teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth'
EVAL_DATASET_SIZE = 5000
CALIBRATION_DATASET_SIZE = 1000
BATCH_SIZE = 64

ROOT_DATASET_DIR = '/teamspace/studios/this_studio/COCO'
IMAGES_DIR = os.path.join(ROOT_DATASET_DIR, 'images')
ANNOTATIONS_JSON_PATH = os.path.join(ROOT_DATASET_DIR, 'annotations/instances_val2017.json')
# ANNOTATIONS_JSON_PATH = "/home/shayaan/Desktop/aimet/my_mmdet/temp.json"

model = DetInferencer(model=CONFIG_PATH, weights=WEIGHTS_PATH, device=DEVICE)
evalDataset = CustomImageDataset(images_dir=IMAGES_DIR, annotations_json_path=ANNOTATIONS_JSON_PATH, transform=transform)
eval_data_loader = DataLoader(evalDataset, batch_size=BATCH_SIZE)
calibration_images = read_txt('/teamspace/studios/this_studio/aimet/Examples/torch/quantization/calibration_image_ids.txt')
calibration_data_loader = DataLoader(calibration_images, batch_size=BATCH_SIZE)

from tqdm.notebook import tqdm
import torch

from mmdet.models.utils import samplelist_boxtype2tensor
from mmengine.registry import MODELS
from mmcv.transforms import Compose

test_evaluator = model.cfg.test_evaluator
test_evaluator.type = 'mmdet.evaluation.CocoMetric' 
test_evaluator.dataset_meta = model.model.dataset_meta
test_evaluator.ann_file = ANNOTATIONS_JSON_PATH
test_evaluator = Compose(test_evaluator)

collate_preprocessor = model.preprocess
predict_by_feat = model.model.bbox_head.predict_by_feat
rescale = True

preprocessor = MODELS.build(model.cfg.model.data_preprocessor)
def add_pred_to_datasample(data_samples, results_list):
    for data_sample, pred_instances in zip(data_samples, results_list):
        data_sample.pred_instances = pred_instances
    samplelist_boxtype2tensor(data_samples)
    return data_samples

DEVICE

[2024-09-24 14:12:21,781] [WARNING] [real_accelerator.py:162:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.
[2024-09-24 14:12:21,784] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cpu (auto detect)
Loads checkpoint by local backend from path: /teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std

09/24 14:12:27 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.


/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


loading annotations into memory...
Done (t=1.53s)
creating index...
index created!


device(type='cpu')

In [4]:
onnx_model_path = "/teamspace/studios/this_studio/aimet/exported_models/bn_folded_int8_embedded/rtm_det_embedded.onnx"

# load onnx model
onnx_model = onnx.load(onnx_model_path)
# get the model's input and output names
input_names = [input.name for input in onnx_model.graph.input]
output_names = [output.name for output in onnx_model.graph.output]
# print the input and output names
print("Input names:", input_names)
print("Output names:", output_names)

Input names: ['input_tensor.1']
Output names: ['1965', '2085', '2203', '2027', '2147', '2266']


In [5]:
modules_to_ignore = ['backbone.stage2.1.blocks.0.conv2.depthwise_conv', 'backbone.stage1.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_7', 'backbone.stage2.1.blocks.0.conv2.pointwise_conv.conv', 'backbone.stage3.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_21', 'backbone.stage4.2.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_30', 'neck.top_down_blocks.0.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_37', 'neck.top_down_blocks.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_44', 'neck.bottom_up_blocks.0.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_51', 'neck.bottom_up_blocks.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_58']
modules_to_ignore

modules = ["module_batch_norm_14", "module_batch_norm_7", "module_batch_norm_21", "module_batch_norm_30", "module_batch_norm_37", "module_batch_norm_44", "module_batch_norm_51", "module_batch_norm_58"]

In [8]:
embdedded_modules = ["/conv/_module_to_wrap_14/Conv", "/conv_14/QuantizeLinear_1", "/conv_14/DequantizeLinear_1", "/conv/_module_to_wrap_7/Conv", "/conv_7/QuantizeLinear_1", "/conv_7/DequantizeLinear_1", "/conv/_module_to_wrap_21/Conv", "/conv_21/QuantizeLinear_1", "/conv_21/DequantizeLinear_1", "/conv/_module_to_wrap_30/Conv", "/conv_30/QuantizeLinear_1", "/conv_30/DequantizeLinear_1", "/conv/_module_to_wrap_37/Conv", "/conv_37/QuantizeLinear_1", "/conv_37/DequantizeLinear_1", "/conv/_module_to_wrap_44/Conv", "/conv_44/QuantizeLinear_1", "/conv_44/DequantizeLinear_1", "/conv/_module_to_wrap_51/Conv", "/conv_51/QuantizeLinear_1", "/conv_51/DequantizeLinear_1", "/conv/_module_to_wrap_58/Conv", "/conv_58/QuantizeLinear_1", "/conv_58/DequantizeLinear_1", "/conv/_module_to_wrap_15/Conv",  "/conv_15/QuantizeLinear_1", "/conv_15/DequantizeLinear_1", ]
len(embdedded_modules)

27

In [6]:
output_path = "./temp.onnx"
b = False
idx = 0
for node in tqdm(onnx_model.graph.node):
    if "/conv/_module_to_wrap_21/Conv" in node.name:
        print(f"{node.name=} {node.output=}")
        b = True
        continue
    if b:
        print(f"{node.name=} {node.output=}")
        idx += 1
        if idx == 4:
            break
# output_names = ["/conv_14/QuantizeLinear_1_output_0"]
# onnx.utils.extract_model(onnx_model_path, output_path, input_names, output_names)

# ort_session = onnxruntime.InferenceSession(output_path, providers=["CPUExecutionProvider"])
# output_name = ort_session.get_outputs()[0].name
# input_name = ort_session.get_inputs()[0].name

  0%|          | 0/1723 [00:00<?, ?it/s]

node.name='/conv/_module_to_wrap_21/Conv' node.output=['/conv/_module_to_wrap_21/Conv_output_0']
node.name='/conv_21/Constant_2' node.output=['/conv_21/Constant_2_output_0']
node.name='/conv_21/Constant_3' node.output=['/conv_21/Constant_3_output_0']
node.name='/conv_21/QuantizeLinear_1' node.output=['/conv_21/QuantizeLinear_1_output_0']
node.name='/conv_21/DequantizeLinear_1' node.output=['/conv_21/DequantizeLinear_1_output_0']


In [ ]:
output_path = "./temp.onnx"
obj = {}

image_path = ["/teamspace/studios/this_studio/COCO/images/000000000139.jpg"]

for node in tqdm(onnx_model.graph.node):
    print(node.name)
    if any([m in node.name for m in modules_to_ignore]):
        print(node.output[0])
        output_names = node.output
        onnx.utils.extract_model(onnx_model_path, output_path, input_names, output_names)
        
        ort_session = onnxruntime.InferenceSession(output_path)
        output_name = ort_session.get_outputs()[0].name
        input_name = ort_session.get_inputs()[0].name
        
        # pre_processed = collate_preprocessor(inputs=image_path, batch_size=1)
        # _, data = list(pre_processed)[0]
        # data = preprocessor(data, False)
        # input_data = data['inputs'].numpy()

        # outputs = ort_session.run([output_name], {input_name: input_data})[0]
        # obj[node.output[0]] = outputs
        # break
    
print(obj)

In [ ]:
import onnx
import onnxruntime
import torch

input_path = onnx_model_path
output_path = "./temp.onnx"
output_names = ["/module_batch_norm_14/BatchNormalization_output_0"]

onnx.utils.extract_model(input_path, output_path, input_names, output_names)

temp_onnx_model = onnx.load("./temp.onnx")
# run inference
ort_session = onnxruntime.InferenceSession("./temp.onnx")
# get the output name
output_name = ort_session.get_outputs()[0].name
# get the input name
input_name = ort_session.get_inputs()[0].name
input_data = torch.randn(1, 3, 640, 640).numpy()

ort_session.run([output_name], {input_name: input_data})